In [0]:
%matplotlib inline
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import os
from tensorflow.keras.applications import MobileNet, DenseNet121, MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import RMSprop, Adagrad, Adam, SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense
from tensorflow.keras.callbacks import CSVLogger
from skimage.io import imread
from skimage.transform import resize


GOOGLE_COLAB = True
TRAINING_LOGS_FILE = "training_logs.csv"
MODEL_SUMMARY_FILE = "model_summary.txt"
MODEL_FILE = "model.h5"

if GOOGLE_COLAB:
    !pip install livelossplot
from livelossplot import PlotLossesKeras


training_data_dir = "/content/training_set" # 8000
validation_data_dir = "/content/validation_set" # 2000
test_data_dir = "/content/test_set" # 7


# Globalparams
IMAGE_SIZE = 224
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
EPOCHS = 10
BATCH_SIZE = int(EPOCHS//2)

input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 3)

In [0]:
model = Sequential()
main_model = MobileNet(weights='imagenet', include_top=False, input_shape=input_shape)
model.add(main_model)
model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(2, activation='sigmoid'))

    
model.compile(loss='binary_crossentropy',
            optimizer=Adam(lr=0.0001, amsgrad=True),
            metrics=['accuracy'])

with open(MODEL_SUMMARY_FILE,"w") as fh:
    model.summary(print_fn=lambda line: fh.write(line + "\n"))

In [0]:
training_data_generator = ImageDataGenerator(
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    rescale=1/255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest',
    validation_split=0.2)

validation_data_generator = ImageDataGenerator(
    rescale=1./255)


training_generator = training_data_generator.flow_from_directory(
    training_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")
validation_generator = validation_data_generator.flow_from_directory(
    validation_data_dir,
    target_size=(IMAGE_WIDTH, IMAGE_HEIGHT),
    batch_size=BATCH_SIZE,
    class_mode="categorical")

In [0]:

model.fit_generator(
    training_generator,
    steps_per_epoch=len(training_generator.filenames)//BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=validation_generator,
    validation_steps=len(validation_generator.filenames)//BATCH_SIZE,
    callbacks=[PlotLossesKeras(), CSVLogger(TRAINING_LOGS_FILE,
                                            append=False,
                                            separator=";")], 
    verbose=1)
model.save(MODEL_FILE)

In [0]:
class_labels = {v: k for k, v in training_generator.class_indices.items()}

print(class_labels)

for file in os.listdir(test_data_dir):
    image_path = os.path.join(test_data_dir, file)
    img = mpimg.imread(image_path)
    img = resize(img,(IMAGE_SIZE,IMAGE_SIZE)) 
    img = np.expand_dims(img,axis=0) 
    prediction = model.predict_classes(img)
    img2 = mpimg.imread(image_path) 
    plt.imshow(img2)
    if prediction[0]==0:
        plt.title("Cheetah")
    elif prediction[0]==1:
         plt.title('Jaguar')
    elif prediction[0]==2:
         plt.title('Leopard')
    elif prediction[0]==3:
         plt.title('Ocelot')
    plt.show()